In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,184 kB]


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-06 03:34:36--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.07MB/s    in 0.2s    

2021-06-06 03:34:37 (6.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ml").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
# Read in data from S3 Buckets for patients
from pyspark import SparkFiles
url = "https://yang5714-bucket.s3.amazonaws.com/patients.csv"
spark.sparkContext.addFile(url)
patient_df = spark.read.csv(SparkFiles.get("patients.csv"), sep=",", header=True)

In [5]:
patient_df.show()

+----------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+------+
|patient_id|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|target|
+----------+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+------+
|         0| 52|  1|  0|     125| 212|  0|      1|    168|    0|    1.0|    2|  2|     0|
|         1| 53|  1|  0|     140| 203|  1|      0|    155|    1|    3.1|    0|  0|     0|
|         2| 70|  1|  0|     145| 174|  0|      1|    125|    1|    2.6|    0|  0|     0|
|         3| 61|  1|  0|     148| 203|  0|      1|    161|    0|    0.0|    2|  1|     0|
|         4| 62|  0|  0|     138| 294|  1|      1|    106|    0|    1.9|    1|  3|     0|
|         5| 58|  0|  0|     100| 248|  0|      0|    122|    0|    1.0|    1|  0|     1|
|         6| 58|  1|  0|     114| 318|  0|      2|    140|    0|    4.4|    0|  3|     0|
|         7| 55|  1|  0|     160| 289|  0|      0|    145|    1|    0.8|    1|  1|     0|
|         

In [6]:
# Read in data from S3 Buckets for chest_pain_type
from pyspark import SparkFiles
url = "https://yang5714-bucket.s3.amazonaws.com/chest_pain_types.csv"
spark.sparkContext.addFile(url)
cp_df = spark.read.csv(SparkFiles.get("chest_pain_types.csv"), sep=",", header=True)

In [7]:
cp_df.show()

+---------------+
|chest_pain_type|
+---------------+
|              0|
|              1|
|              2|
|              3|
+---------------+



In [8]:
# Read in data from S3 Buckets for restecg_types
from pyspark import SparkFiles
url = "https://yang5714-bucket.s3.amazonaws.com/restecg_types.csv"
spark.sparkContext.addFile(url)
restecg_df = spark.read.csv(SparkFiles.get("restecg_types.csv"), sep=",", header=True)

In [9]:
restecg_df.show()

+------------------------------------+
|resting_electrocardiographic_results|
+------------------------------------+
|                                   0|
|                                   1|
|                                   2|
+------------------------------------+



In [10]:
# Read in data from S3 Buckets for vessel_color
from pyspark import SparkFiles
url = "https://yang5714-bucket.s3.amazonaws.com/vessel_color.csv"
spark.sparkContext.addFile(url)
vessel_df = spark.read.csv(SparkFiles.get("vessel_color.csv"), sep=",", header=True)

In [11]:
vessel_df.show()

+------------+
|vessel_color|
+------------+
|           0|
|           1|
|           2|
|           3|
+------------+



In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://heart-database.cuuvdeqkecxo.us-east-1.rds.amazonaws.com:5432/heart_disease"
config = {"user":"postgres",
          "password": "12345678",
          "driver":"org.postgresql.Driver"}

In [15]:
# Write DataFrame to patient table in RDS
patient_df.write.jdbc(url=jdbc_url, table='patient', mode=mode, properties=config)

In [16]:
# Write DataFrame to chest_pain table in RDS
cp_df.write.jdbc(url=jdbc_url, table='chest_pain', mode=mode, properties=config)

In [17]:
# Write DataFrame to resting_electrocardiographic_results table in RDS
restecg_df.write.jdbc(url=jdbc_url, table='restecg', mode=mode, properties=config)

In [18]:
# Write DataFrame to vessel_color table in RDS
vessel_df.write.jdbc(url=jdbc_url, table='vessel_color', mode=mode, properties=config)